In [1]:
!pip install transformers
!pip install datasets
!pip install sentencepiece
!pip install torch

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import BertForSequenceClassification, BertTokenizer
from transformers import Trainer, TrainingArguments
from transformers import pipeline
from datasets import Dataset

In [15]:
df = pd.read_csv('/content/re_dataset.csv', encoding='latin1')

In [16]:
df.head()

,Tweet,HS,Abusive,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other,HS_Weak,HS_Moderate,HS_Strong
0,- disaat semua cowok berusaha melacak perhatia...,1,1,1,0,0,0,0,0,1,1,0,0
1,RT USER: USER siapa yang telat ngasih tau elu?...,0,1,0,0,0,0,0,0,0,0,0,0
2,"41. Kadang aku berfikir, kenapa aku tetap perc...",0,0,0,0,0,0,0,0,0,0,0,0
3,USER USER AKU ITU AKU\n\nKU TAU MATAMU SIPIT T...,0,0,0,0,0,0,0,0,0,0,0,0
4,USER USER Kaum cebong kapir udah keliatan dong...,1,1,0,1,1,0,0,0,0,0,1,0


In [17]:
df.drop(['Abusive','HS_Individual','HS_Group',
         'HS_Religion','HS_Race','HS_Physical',
         'HS_Gender','HS_Other','HS_Weak',
         'HS_Moderate','HS_Strong'],axis=1, inplace=True)
df.head()
df.head()

,Tweet,HS
0,- disaat semua cowok berusaha melacak perhatia...,1
1,RT USER: USER siapa yang telat ngasih tau elu?...,0
2,"41. Kadang aku berfikir, kenapa aku tetap perc...",0
3,USER USER AKU ITU AKU\n\nKU TAU MATAMU SIPIT T...,0
4,USER USER Kaum cebong kapir udah keliatan dong...,1


In [18]:
df['HS'].value_counts()

,count
HS,
0,7608
1,5561


In [19]:
train_df, test_df = train_test_split (df,test_size=0.2, random_state=42)
train_df, val_df = train_test_split (train_df, test_size=0.1, random_state=42)

print (f"Train size: {len(train_df)}")
print (f"Validation size: {len(val_df)}")
print (f"Test size: {len(test_df)}")

Train size: 9481
Validation size: 1054
Test size: 2634


In [20]:
model_name = 'indolem/indobert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/234k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indolem/indobert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
def tokenize_function(examples):
    return tokenizer(examples['Tweet'], padding="max_length", truncation=True, max_length=128)

train_dataset = Dataset.from_pandas(train_df.rename(columns={'HS': 'labels'}))
val_dataset = Dataset.from_pandas(val_df.rename(columns={'HS': 'labels'}))
test_dataset = Dataset.from_pandas(test_df.rename(columns={'HS': 'labels'}))

tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_val = val_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/9481 [00:00<?, ? examples/s]

Map:   0%|          | 0/1054 [00:00<?, ? examples/s]

Map:   0%|          | 0/2634 [00:00<?, ? examples/s]

In [22]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
)

trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: firazzzaf11 (firazzzaf11-stt-wastukancana) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,0.426100,0.342284
2,0.213200,0.405144
3,0.062800,0.421882


TrainOutput(global_step=1779, training_loss=0.3421995245452139, metrics={'train_runtime': 769.9075, 'train_samples_per_second': 36.943, 'train_steps_per_second': 2.311, 'total_flos': 1870916936901120.0, 'train_loss': 0.3421995245452139, 'epoch': 3.0})

In [23]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _=precision_recall_fscore_support(labels,preds,average='binary')
    acc = accuracy_score (labels, preds)
    return {
    'accuracy':acc,
    'f1':f1,
    'precision':precision,
    'recall':recall
}

trainer = Trainer(
    model=model,
    args=TrainingArguments(output_dir='./results', per_device_eval_batch_size=16),
    compute_metrics=compute_metrics,
)

result = trainer.evaluate(tokenized_test)
print(result)

{'eval_loss': 0.34738296270370483, 'eval_model_preparation_time': 0.0026, 'eval_accuracy': 0.8557327258921792, 'eval_f1': 0.8264840182648402, 'eval_precision': 0.8442164179104478, 'eval_recall': 0.8094812164579607, 'eval_runtime': 20.2023, 'eval_samples_per_second': 130.381, 'eval_steps_per_second': 8.167}


In [24]:
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)

def predict_hate_speech(text):
    result=classifier(text)
    label = "Hate Speech" if result[0]['label']=="LABEL_1" else "Non-Hate Speech"
    confidence = result[0]['score']
    print(f"Text: {text}")
    print(f"Prediction: {label} (Confidence: {confidence:.2f})")
    print("---")

test_texts = [
   "Dasar kamu bodoh dan tidak berguna!",
   "Saya suka makan nasi goreng",
   "Kelompok itu harus diusir dari sinil",
   "Terima kasih atas bantuannya",
   "Dia memang pantas mendapatkan itu, dasar sampah masyarakat!"
]

for text in test_texts:
    predict_hate_speech(text)

Device set to use cuda:0


Text: Dasar kamu bodoh dan tidak berguna!
Prediction: Hate Speech (Confidence: 0.89)
---
Text: Saya suka makan nasi goreng
Prediction: Non-Hate Speech (Confidence: 0.97)
---
Text: Kelompok itu harus diusir dari sinil
Prediction: Non-Hate Speech (Confidence: 0.55)
---
Text: Terima kasih atas bantuannya
Prediction: Non-Hate Speech (Confidence: 0.98)
---
Text: Dia memang pantas mendapatkan itu, dasar sampah masyarakat!
Prediction: Hate Speech (Confidence: 0.98)
---
